In [ ]:
!pip install wandb 

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import  Conv2D,MaxPool2D,Flatten,Dense

import wandb
from wandb.keras import WandbCallback

import numpy as np
import cv2

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
wandb.init(project="Clothes Recognizer")

## Dataset

In [ ]:
dataset_path = "/content/drive/MyDrive/Clothes Dataset"

width = height = 224

idg = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    rotation_range = 15,
    zoom_range = 0.1,
    brightness_range =  (0.9, 1.1), 
    validation_split = 0.2
)

train_data = idg.flow_from_directory(
    dataset_path,
    target_size = (width , height),
    class_mode = 'categorical',
    subset = 'training'
)

val_data = idg.flow_from_directory(
    dataset_path,
    target_size = (width , height),
    class_mode = 'categorical',
    subset = 'validation'
)

## Model

In [6]:
model = tf.keras.models.Sequential([
    Conv2D(32 , (3,3), activation='relu', input_shape=(width, height, 3)),               
    Conv2D(32 , (3,3), activation='relu'),
    MaxPool2D(),
    Conv2D(32 , (3,3), activation='relu'),
    Conv2D(32 , (3,3), activation='relu'),
    MaxPool2D(),
    Conv2D(64 , (3,3), activation='relu'),
    Conv2D(64 , (3,3), activation='relu'),
    MaxPool2D(),

    Flatten(),
    Dense(256, activation='relu'),
    Dense(4, activation='softmax')
])

In [7]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss = tf.keras.losses.categorical_crossentropy,
              metrics =['accuracy'] )

## Train

In [ ]:
model.fit(train_data, validation_data= val_data, epochs=10,callbacks=[WandbCallback()])

## Test

In [9]:
dataset_path = "/content/drive/MyDrive/Clothes Test"

width = height = 224

idg = ImageDataGenerator(
    rescale = 1./255
)

test_data = idg.flow_from_directory(
    dataset_path,
    target_size = (width , height),
    class_mode = 'categorical'
)

Found 84 images belonging to 4 classes.


In [10]:
model.evaluate(test_data)

3/3 [==============================] - 13s 6s/step - loss: 1.1335 - accuracy: 0.6071


[1.1335300207138062, 0.6071428656578064]

## Save Model

In [11]:
model.save("Recognizer.h5")

## Inference

In [16]:
img = cv2.imread("/content/3.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (width, height))
img = img / 255.0
img = img.reshape(1, width, height, 3)

result = model.predict(img)
result
pred = np.argmax(result)
pred
if pred == 0:
  print("👜")
elif pred == 1:
  print("👠")
elif pred == 2:
  print("👟")
elif pred == 3:
  print("🧥")

👠
